# Building a RAG application from scratch

Here is a high-level overview of the system we want to build:

<img src='images/system1.png' width="1200">

Let's start by loading the environment variables we need to use.

In [54]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# [1hr Talk] Intro to Large Language Models - Andrej Karpathy
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=zjkBMFhNj_g"

## Setting up the model
Let's define the LLM model that we'll use as part of the workflow.

In [18]:
# langchain supports multiple language models
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

We can test the model by asking a simple question.

In [19]:
model.invoke("What MLB team won the World Series during the COVID-19 pandemic?")

AIMessage(content='The Los Angeles Dodgers won the World Series during the COVID-19 pandemic in 2020. They defeated the Tampa Bay Rays in a six-game series to claim their first championship since 1988.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

The result from the model is an `AIMessage` instance containing the answer. We can extract this answer by chaining the model with an [output parser](https://python.langchain.com/docs/modules/model_io/output_parsers/).

Here is what chaining the model with an output parser looks like:

<img src='images/chain1.png' width="1200">

For this example, we'll use a simple `StrOutputParser` to extract the answer as a string.

In [20]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
# format the output to only include the string in content
chain = model | parser
chain.invoke("What MLB team won the World Series during the COVID-19 pandemic?")

'The Los Angeles Dodgers won the World Series during the COVID-19 pandemic in 2020.'

## Introducing prompt templates

We want to provide the model with some context and the question. [Prompt templates](https://python.langchain.com/docs/modules/model_io/prompts/quick_start) are a simple way to define and reuse prompts.

In [21]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Mary's sister is Susana", question="Who is Mary's sister?")

'Human: \nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Mary\'s sister is Susana\n\nQuestion: Who is Mary\'s sister?\n'

We can now chain the prompt with the model and the output parser.

<img src='images/chain2.png' width="1200">

In [22]:
chain = prompt | model | parser
chain.invoke({
    "context": "Mary's sister is Susana",
    "question": "Who is Mary's sister?"
})

'Susana'

## Combining chains

We can combine different chains to create more complex workflows. For example, let's create a second chain that translates the answer from the first chain into a different language.

Let's start by creating a new prompt template for the translation chain:

In [23]:
translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

We can now create a new translation chain that combines the result from the first chain with the translation prompt.

Here is what the new workflow looks like:

<img src='images/chain3.png' width="1200">

In [24]:
from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "Mary's sister is Susana. She doesn't have any more siblings.",
        "question": "How many sisters does Mary have?",
        "language": "Spanish",
    }
)

'María tiene una hermana, Susana.'

## Transcribing the YouTube Video

The context we want to send the model comes from a YouTube video. Let's download the video and transcribe it using [OpenAI's Whisper](https://openai.com/research/whisper).

In [25]:
import tempfile
import whisper
from pytube import YouTube


# Let's do this only if we haven't created the transcription file yet.
if not os.path.exists("transcription.txt"):
    youtube = YouTube(YOUTUBE_VIDEO)
    audio = youtube.streams.filter(only_audio=True).first()

    # Let's load the base model. This is not the most accurate
    # model but it's fast.
    whisper_model = whisper.load_model("base")

    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()

        with open("transcription.txt", "w") as file:
            file.write(transcription)

Let's read the transcription and display the first few characters to ensure everything works as expected.

In [26]:
with open("transcription.txt") as file:
    transcription = file.read()

transcription[:100]

'Hi everyone. So recently I gave a 30 minute talk on large language models just kind of like an intro'

## Using the entire transcription as context

If we try to invoke the chain using the transcription as context, the model will return an error because the context is too long.

Large Language Models support limitted context sizes. The video we are using is too long for the model to handle, so we need to find a different solution.

In [27]:
try:
    chain.invoke({
        "context": transcription,
        "question": "Is reading papers a good idea?"
    })
except Exception as e:
    print(e)

## Splitting the transcription

Since we can't use the entire transcription as the context for the model, a potential solution is to split the transcription into smaller chunks. We can then invoke the model using only the relevant chunks to answer a particular question:

<img src='images/system2.png' width="1200">

Let's start by loading the transcription in memory:

In [28]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("transcription.txt")
text_documents = loader.load()
text_documents

[Document(page_content="Hi everyone. So recently I gave a 30 minute talk on large language models just kind of like an intro talk Unfortunately that talk was not recorded But a lot of people came to me after the talk and they told me that they really liked the talk So I would just I thought I was just re-recorded and basically put it up on YouTube So here we go the busy persons intro to large language models director scut Okay, so let's begin First of all, what is a large language model really? Well a large language model is just two files, right? Um, there will be two files in this hypothetical directory So for example, where can we do the specific example of the llama to 70b model? This is a large language model released by meta AI And this is basically the llama series of language models the second iteration of it And this is the 70 billion parameter model of Of this series. So there's multiple models belong to the llama to series 7 billion 13 billion 34 billion and 70 billion as th

There are many different ways to split a document. For this example, we'll use a simple splitter that splits the document into chunks of a fixed size. Check [Text Splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/) for more information about different approaches to splitting documents.

For illustration purposes, let's split the transcription into chunks of 100 characters with an overlap of 20 characters and display the first few chunks:

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(text_documents)[:5]

[Document(page_content='Hi everyone. So recently I gave a 30 minute talk on large language models just kind of like an intro', metadata={'source': 'transcription.txt'}),
 Document(page_content='of like an intro talk Unfortunately that talk was not recorded But a lot of people came to me after', metadata={'source': 'transcription.txt'}),
 Document(page_content='came to me after the talk and they told me that they really liked the talk So I would just I', metadata={'source': 'transcription.txt'}),
 Document(page_content='So I would just I thought I was just re-recorded and basically put it up on YouTube So here we go', metadata={'source': 'transcription.txt'}),
 Document(page_content="So here we go the busy persons intro to large language models director scut Okay, so let's begin", metadata={'source': 'transcription.txt'})]

For our specific application, let's use 1000 characters instead:

In [30]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)
documents[:5]

[Document(page_content="Hi everyone. So recently I gave a 30 minute talk on large language models just kind of like an intro talk Unfortunately that talk was not recorded But a lot of people came to me after the talk and they told me that they really liked the talk So I would just I thought I was just re-recorded and basically put it up on YouTube So here we go the busy persons intro to large language models director scut Okay, so let's begin First of all, what is a large language model really? Well a large language model is just two files, right? Um, there will be two files in this hypothetical directory So for example, where can we do the specific example of the llama to 70b model? This is a large language model released by meta AI And this is basically the llama series of language models the second iteration of it And this is the 70 billion parameter model of Of this series. So there's multiple models belong to the llama to series 7 billion 13 billion 34 billion and 70 billion as th

## Finding the relevant chunks

Given a particular question, we need to find the relevant chunks from the transcription to send to the model. Here is where the idea of **embeddings** comes into play.

An embedding is a mathematical representation of the semantic meaning of a word, sentence, or document. It's a projection of a concept in a high-dimensional space. Embeddings have a simple characteristic: The projection of related concepts will be close to each other, while concepts with different meanings will lie far away. You can use the [Cohere's Embed Playground](https://dashboard.cohere.com/playground/embed) to visualize embeddings in two dimensions.

To provide with the most relevant chunks, we can use the embeddings of the question and the chunks of the transcription to compute the similarity between them. We can then select the chunks with the highest similarity to the question and use them as the context for the model:

<img src='images/system3.png' width="1200">

Let's generate embeddings for an arbitrary query:

In [31]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embedded_query = embeddings.embed_query("Who is Mary's sister?")

print(f"Embedding length: {len(embedded_query)}")
print(embedded_query[:10])

Embedding length: 1536
[-0.001371190081765891, -0.03434698236453119, -0.011476094990116788, 0.0012773800454156574, -0.026166747008526288, 0.009230907949392044, -0.015660022937300136, 0.0017948988196774898, -0.011851335135517721, -0.03324627818637449]


To illustrate how embeddings work, let's first generate the embeddings for two different sentences:

In [32]:
sentence1 = embeddings.embed_query("Mary's sister is Susana")
sentence2 = embeddings.embed_query("Pedro's mother is a teacher")

We can now compute the similarity between the query and each of the two sentences. The closer the embeddings are, the more similar the sentences will be.

We can use [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity) to calculate the similarity between the query and each of the sentences:

In [33]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(0.9174548954382715, 0.7680495517171395)

## Setting up a Vector Store

We need an efficient way to store document chunks, their embeddings, and perform similarity searches at scale. To do this, we'll use a **vector store**.

A vector store is a database of embeddings that specializes in fast similarity searches. 

<img src='images/system4.png' width="1200">

To understand how a vector store works, let's create one in memory and add a few embeddings to it:

In [34]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "Mary's sister is Susana",
        "John and Tommy are brothers",
        "Patricia likes white cars",
        "Pedro's mother is a teacher",
        "Lucia drives an Audi",
        "Mary has two siblings",
    ],
    embedding=embeddings,
)

We can now query the vector store to find the most similar embeddings to a given query:

In [35]:
vectorstore1.similarity_search_with_score(query="Who is Mary's sister?", k=3)

[(Document(page_content="Mary's sister is Susana"), 0.9174549036927806),
 (Document(page_content='Mary has two siblings'), 0.9046656304803649),
 (Document(page_content='John and Tommy are brothers'), 0.8012829892542408)]

## Connecting the vector store to the chain

We can use the vector store to find the most relevant chunks from the transcription to send to the model. Here is how we can connect the vector store to the chain:

<img src='images/chain4.png' width="1200">

We need to configure a [Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/). The retriever will run a similarity search in the vector store and return the most similar documents back to the next step in the chain.

We can get a retriever directly from the vector store we created before: 

In [36]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("Who is Mary's sister?")

[Document(page_content="Mary's sister is Susana"),
 Document(page_content='Mary has two siblings'),
 Document(page_content='John and Tommy are brothers'),
 Document(page_content="Pedro's mother is a teacher")]

Our prompt expects two parameters, "context" and "question." We can use the retriever to find the chunks we'll use as the context to answer the question.

We can create a map with the two inputs by using the [`RunnableParallel`](https://python.langchain.com/docs/expression_language/how_to/map) and [`RunnablePassthrough`](https://python.langchain.com/docs/expression_language/how_to/passthrough) classes. This will allow us to pass the context and question to the prompt as a map with the keys "context" and "question."

In [37]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup.invoke("What color is Patricia's car?")

{'context': [Document(page_content='Patricia likes white cars'),
  Document(page_content='Lucia drives an Audi'),
  Document(page_content="Pedro's mother is a teacher"),
  Document(page_content="Mary's sister is Susana")],
 'question': "What color is Patricia's car?"}

Let's now add the setup map to the chain and run it:



In [38]:
chain = setup | prompt | model | parser
chain.invoke("What color is Patricia's car?")

'White'

Let's invoke the chain using another example:

In [39]:
chain.invoke("What car does Lucia drive?")

'Lucia drives an Audi.'

## Loading transcription into the vector store

We initialized the vector store with a few random strings. Let's create a new vector store using the chunks from the video transcription.

In [40]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

Let's set up a new chain using the correct vector store. This time we are using a different equivalent syntax to specify the [`RunnableParallel`](https://python.langchain.com/docs/expression_language/how_to/map) portion of the chain:

In [42]:
chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("What does this video talk about?")

'The video talk is about large language models, their capabilities, challenges, and future potential.'

In [43]:
chain.invoke("What is large language model?")

'A large language model is a model with a large number of parameters used for tasks like next word prediction.'

In [44]:
chain.invoke("What are the limitations of large language models?")

"Large language models have limitations in that they operate in a system one setting, where they can't think and reason through possibilities like a tree. They simply generate the next word based on the input sequence."

## Setting up Pinecone

So far we've used an in-memory vector store. In practice, we need a vector store that can handle large amounts of data and perform similarity searches at scale. For this example, we'll use [Pinecone](https://www.pinecone.io/).

The first step is to create a Pinecone account, set up an index, get an API key, and set it as an environment variable `PINECONE_API_KEY`.

Then, we can load the transcription documents into Pinecone:

In [55]:
from langchain_pinecone import Pinecone, PineconeVectorStore

index_name = "youtube-rag"

pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)


Let's now run a similarity search on pinecone to make sure everything works:

In [56]:
pinecone.similarity_search("What does this video talk about?")[:3]

[Document(page_content="achieved, how they're trained. I talked about the promise of language models and where they are headed in the future. And I've also talked about the challenges of this new and emerging paradigm of computing and a lot of ongoing work and certainly a very exciting space to keep track of. Bye.", metadata={'source': 'transcription.txt'}),
 Document(page_content="Hi everyone. So recently I gave a 30 minute talk on large language models just kind of like an intro talk Unfortunately that talk was not recorded But a lot of people came to me after the talk and they told me that they really liked the talk So I would just I thought I was just re-recorded and basically put it up on YouTube So here we go the busy persons intro to large language models director scut Okay, so let's begin First of all, what is a large language model really? Well a large language model is just two files, right? Um, there will be two files in this hypothetical directory So for example, where can 

Let's setup the new chain using Pinecone as the vector store:

In [57]:
chain = (
    {"context": pinecone.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

chain.invoke("What does this video talk about?")

'Large language models, their challenges, and the ongoing work in the field.'